In [6]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [7]:
class DNN(Model):
    def __init__(self,hidden_dim,output_dim):
        super().__init__()
        self.l1=Dense(hidden_dim,activation='relu')
        self.d1=Dropout(0.5)
        self.l2=Dense(hidden_dim,activation='relu')
        self.d2=Dropout(0.5)
        self.l3=Dense(hidden_dim,activation='relu')
        self.d3=Dropout(0.5)
        self.l4=Dense(output_dim,activation='softmax')
        self.ls=[self.l1,self.d1,self.l2,self.d2,self.l3,self.d3,self.l4]
        
    def call(self,x):
        for layer in self.ls:
            x=layer(x)
        return x

In [8]:
mnist=datasets.mnist
(x_train,t_train),(x_test,t_test)=mnist.load_data()

x_train=(x_train.reshape(-1,784)/255).astype(np.float32)
x_test=(x_test.reshape(-1,784)/255).astype(np.float32)
t_train=np.eye(10)[t_train].astype(np.float32)
t_test=np.eye(10)[t_test].astype(np.float32)

In [9]:
model=DNN(200,10)

In [10]:
criterion=losses.CategoricalCrossentropy()
optimizer=optimizers.SGD(learning_rate=0.1)
train_loss=metrics.Mean()
train_acc=metrics.CategoricalAccuracy()

def compute_loss(t,y):
    return criterion(t,y)

def train_step(x,t):
    with tf.GradientTape() as tape:
        preds=model(x)
        loss=compute_loss(preds,t)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss)
    train_acc(t,preds)
    return loss

epochs=30
batch_size=100
n_batches=x_train.shape[0]//batch_size

for epoch in range(epochs):
    x_,t_=shuffle(x_train,t_train)
    
    for batch in range(n_batches):
        start=batch*batch_size
        end=start+batch_size
        train_step(x_[start:end],t_[start:end])
    
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}'.format(epoch,train_loss.result(),train_acc.result()))


epoch: 0, loss: 2.661, acc: 0.839
epoch: 1, loss: 1.910, acc: 0.884
epoch: 2, loss: 1.634, acc: 0.900
epoch: 3, loss: 1.492, acc: 0.909
epoch: 4, loss: 1.431, acc: 0.912
epoch: 5, loss: 1.429, acc: 0.912
epoch: 6, loss: 1.450, acc: 0.911
epoch: 7, loss: 1.605, acc: 0.901
epoch: 8, loss: 1.907, acc: 0.882
epoch: 9, loss: 2.223, acc: 0.863
epoch: 10, loss: 3.118, acc: 0.807
epoch: 11, loss: 4.023, acc: 0.751
epoch: 12, loss: 4.830, acc: 0.701
epoch: 13, loss: 5.522, acc: 0.658
epoch: 14, loss: 6.122, acc: 0.621
epoch: 15, loss: 6.646, acc: 0.588
epoch: 16, loss: 7.109, acc: 0.559
epoch: 17, loss: 7.521, acc: 0.534
epoch: 18, loss: 7.889, acc: 0.511
epoch: 19, loss: 8.221, acc: 0.490
epoch: 20, loss: 8.520, acc: 0.472
epoch: 21, loss: 8.793, acc: 0.455
epoch: 22, loss: 9.042, acc: 0.439
epoch: 23, loss: 9.270, acc: 0.425
epoch: 24, loss: 9.480, acc: 0.412
epoch: 25, loss: 9.674, acc: 0.400
epoch: 26, loss: 9.853, acc: 0.389
epoch: 27, loss: 10.020, acc: 0.379
epoch: 28, loss: 10.175, acc:

In [11]:
test_loss=metrics.Mean()
test_acc=metrics.CategoricalAccuracy()

def test_step(x,t):
    preds=model(x)
    loss=compute_loss(t,preds)
    test_loss(loss)
    test_acc(t,preds)
    return loss

test_step(x_test,t_test)

print('test_loss: {:.3f}, test_acc:{:.3f}'.format(test_loss.result(),test_acc.result()))

test_loss: 14.455, test_acc:0.103
